Source: [link](https://github.com/cs230-stanford/cs230-code-examples/blob/master/tensorflow/vision/build_dataset.py)
### TODO
- create py script with argparse
- Handle minority classes > 10

In [223]:
import argparse
import random
import os

from  pathlib import Path
from PIL import Image
# from tqdm import tqdm
from tqdm.notebook import tqdm

In [224]:
data_dir = 'test_dataset'
size = 64
seed = 230
splits = [0.7, 0.15, 0.15]
output_dir = "test_dataset_splitted"

random.seed(seed)
assert sum(splits) == 1.0, 'Sum of splits must be 1.'

In [225]:
def resize_and_save(filename, output_dir, size=size):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(os.path.join(output_dir, filename.split('/')[-1]))

In [226]:
class_names = os.listdir(data_dir)
num_classes = len(class_names)
num_samples = len(list(Path(data_dir).glob('*/*')))

image_folders = [os.path.join(data_dir, class_name) for class_name in class_names]

# Create folders for train, test and val with subfolders
train_folders = [os.path.join(output_dir, 'train', class_name) for class_name in class_names]
test_folders = [os.path.join(output_dir, 'test', class_name) for class_name in class_names]
val_folders = [os.path.join(output_dir, 'val', class_name) for class_name in class_names]

# Create all folders
[Path(train_folder).mkdir(parents=True, exist_ok=True) for train_folder in train_folders]
[Path(test_folder).mkdir(parents=True, exist_ok=True) for test_folder in test_folders]
[Path(val_folder).mkdir(parents=True, exist_ok=True) for val_folder in val_folders];

In [228]:
tqdm_img = tqdm(total=num_samples, desc='Images', position=0)

# Iterate over categories
for idx, directory in enumerate(image_folders):
    random.seed(seed)
    # Get filenames in category, sort and shuffle (for reproducible split)
    filenames = os.listdir(directory)
    filenames.sort()
    random.shuffle(filenames)
    samples_in_dir = len(filenames)
    
    # Calculate number of samples for each dataset
    # NB!! Make sure for samples < 10 are distributed correctly
    sample_splits = [int(split*samples_in_dir) for split in splits]
    
    # Split dataset into train test val
    ds = {'train': filenames[:sample_splits[0]],
          'test': filenames[sample_splits[0]:-sample_splits[1]],
          'val': filenames[-sample_splits[2]:]}
    
    # Copy files to correct folder/split
    for split in ds:
        output = os.path.join(output_dir, split, directory.split("/")[-1])
        for filename in ds[split]:
            filename = os.path.join(directory, filename)
            resize_and_save(filename, output, size=size)
            tqdm_img.update(1)

print ("Done building dataset.")

Done building dataset.
